In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam

training_path = 'train/'
test_path = 'test/'
train=pd.read_csv('train.csv')

#print(os.listdir())

In [ ]:
datagen = ImageDataGenerator(rescale=(1/255))
batch_size = 150
train['has_cactus'] = train['has_cactus'].astype(str)
train_generator = datagen.flow_from_dataframe(dataframe=train[:15001],directory=training_path,x_col="id",
                                            y_col="has_cactus",class_mode="binary",batch_size=batch_size,
                                            target_size=(150,150))

valid_generator = datagen.flow_from_dataframe(dataframe = train[15000:],directory=training_path,x_col="id", y_col="has_cactus",
                                             class_mode="binary",batch_size= batch_size ,target_size=(150,150))

batchX, batchy = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [ ]:
for i in range(0,1):
    image = batchX[i]
    image = image.squeeze()
    plt.imshow(image)
    plt.show()

In [ ]:
class TrainModel:
    
    def __init__(self, train_generator, lr: float = 0.001, epochs: int = 10, batch_size: int = 32,
                 loss: str = 'binary_crossentropy'):
        self.train_generator = train_generator
        self.valid_generator = valid_generator
                
        self.epochs = epochs
        self.batch_size = batch_size

        self.optimizer = Adam(lr=lr)
        self.loss = loss

    def create_model(self, summarize: bool = True):
        model= Sequential()
        model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
        model.add(MaxPooling2D((2,2)))
        model.add(Conv2D(filters = 32, kernel_size = 1, activation = 'relu'))
        model.add(Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
        model.add(MaxPooling2D((2,2)))
        model.add(Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
        model.add(MaxPooling2D((2,2)))
        model.add(Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
        model.add(MaxPooling2D((2,2)))
        model.add(Flatten())
        model.add(Dense(512,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        
        if summarize:
            model.summary()

        return model    
    
    def compile(self, kmodel: Sequential):
        kmodel.compile(loss=self.loss,optimizer=self.optimizer,metrics=['acc'])
        
        return kmodel
    
    def train(self, kmodel: Sequential):
        history = kmodel.fit_generator(self.train_generator,steps_per_epoch=100,epochs=10,validation_data=valid_generator,
                            validation_steps=50) 
        return history, kmodel

In [ ]:
def plot_training_curves(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Losses')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, acc, 'r', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Accuracies')
    plt.legend()
    plt.figure()
    
    plt.show()

In [ ]:
trainer = TrainModel(train_generator, epochs=2, batch_size=1024, lr=0.0002)

model = trainer.create_model()
model = trainer.compile(model)

history = trainer.train(model)

history=model.fit_generator(train_generator,steps_per_epoch=5,epochs=10,validation_data=valid_generator,
                            validation_steps=50) 

In [ ]:
plot_training_curves(history)